# Imports

In [2]:
import os
import re
import cv2
import time
import tarfile
import requests
import numpy as np
import pandas as pd
import networkx as nx
import ringity as rng
import urllib.request
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import xml.etree.ElementTree as ET

from collections import Counter
from pathlib import Path
from itertools import combinations, product as iter_product
from skimage.metrics import mean_squared_error as mse
from scipy.spatial.distance import pdist, squareform
from zipfile import ZipFile

rng.set_theme()
DATA_DIR = Path("..") / "data"
EMPIRICALNET_DIR = DATA_DIR / "empirical_networks"
RAWDATA_DIR = DATA_DIR / "raw_data"

In [3]:
ls

biological_networks-extended.ipynb
extended-SFigX-exploring_different_metric_structures.ipynb
Fig1+Fig2F-biological_networks.ipynb
Fig3-network_model_parameters.ipynb
_legacy/
SFigX-exploring_different_metric_structures.ipynb


# Ego-networks

## Combined networks

### Preparation

In [4]:
# ---------------
# Paths and URLs
# ---------------

EGO_FOLDER = os.getcwd() + "/data/ego_network"
EGO_URL    = "https://snap.stanford.edu/data/facebook_combined.txt.gz"
EGO_EDGELIST_FILE = EGO_FOLDER + "/facebook_combined.txt.gz"

In [5]:
# --------------
# Download data
# --------------

os.makedirs(EGO_FOLDER, exist_ok=True)

r = requests.get(EGO_URL, allow_redirects=True)
with open(EGO_EDGELIST_FILE, 'wb') as f:
    f.write(r.content)

### Analysis

In [ ]:
G_ego = nx.read_edgelist(EGO_EDGELIST_FILE)

pos = nx.spring_layout(G_ego)

In [ ]:
rng.plot_nx(G_ego, pos=pos, node_alpha=0.5)

In [ ]:
degree_sequence = sorted([d for n, d in G_ego.degree()])
degreeCount = Counter(degree_sequence)
degs, cnts = zip(*degreeCount.items())

deg_fig, ax = plt.subplots(figsize=(8,6))
deg_fig.patch.set_alpha(0)
ax.patch.set_alpha(0)

ax.bar(degs, cnts, width=1, linewidth=0);

rng.ax_setup(ax)

In [ ]:
degree_sequence = sorted([d for n, d in G_ego.degree()])

deg_fig, ax = plt.subplots(figsize=(8,6))
deg_fig.patch.set_alpha(0)
ax.patch.set_alpha(0)

ax.hist(degree_sequence, bins=25)

rng.ax_setup(ax)

In [ ]:
# ----------------------
# calclulating diagrams
# ----------------------

dgm_ego = rng.diagram(G_ego)

In [ ]:
# --------------------
# persistence visuals
# --------------------

fig, axes = plt.subplots(2, figsize=(7,13))

axes[0].set_title("Ego-network", fontsize=20)

rng.plot_bar(dgm_ego, ax=axes[0], cap=50)
rng.plot_dgm(dgm_ego, ax=axes[1])

axes[0].annotate(f'score = {dgm_ego.score  :.3f}', xy=(0.7, 0.875), xycoords='axes fraction', fontsize=16);
axes[1].annotate(f'score = {dgm_ego.score  :.3f}', xy=(0.7, 0.125), xycoords='axes fraction', fontsize=16);

## Individual networks

### Preparation

In [ ]:
# ---------------
# Paths and URLs
# ---------------

EGO_FOLDER = os.getcwd() + "/data/ego_network"
EGO_URL    = "https://snap.stanford.edu/data/facebook.tar.gz"
EGO_TAR = EGO_FOLDER + "/facebook.tar.gz"

In [ ]:
# --------------
# Download data
# --------------

if not os.path.exists(EGO_FOLDER):
    os.makedirs(EGO_FOLDER)

r = requests.get(EGO_URL, allow_redirects=True)
with open(EGO_TAR, 'wb') as f:
    f.write(r.content)

tf = tarfile.open(EGO_TAR)
tf.extractall(EGO_FOLDER)

### Analysis

In [ ]:
# ---------------
# Set parameters 
# ---------------

min_nodes = 100

In [ ]:
# --------------------------------
# Extract ego-networks
# --------------------------------


for file in os.listdir(f"{EGO_FOLDER}/facebook"):
    if not file.endswith('edges'):
        continue
    
    G = nx.read_edgelist(f"{EGO_FOLDER}/facebook/{file}")
    H = nx.induced_subgraph(G, max(nx.connected_components(G), key=len))
    
    if H.number_of_nodes() < min_nodes:
        continue
    
    dgm = rng.diagram(H)
    
    print('ID: ', file.split('.')[0])
    print('#Nodes in graph: ', len(G))
    print('#Nodes in LCC:   ', len(H))
    print('ring-score:      ', round(dgm.score, 3))
    print()

# Foodweb networks

### Preparation

In [ ]:
FOODWEB_FOLDER = os.getcwd() + "/data/foodweb_networks"

## Bipartite networks

In [ ]:
# ---------------
# Set parameters 
# ---------------

min_nodes = 100

In [ ]:
# -------------------------
# Extract foodweb networks
# -------------------------


for file in os.listdir(FOODWEB_FOLDER):
    if not file.endswith('csv') or file == 'references.csv':
        continue
    
 #  Load table 
    foodweb_matrix = pd.read_csv(f"{FOODWEB_FOLDER}/{file}", index_col=0)
    consumers = foodweb_matrix.index
    resources = foodweb_matrix.columns

 #  Construct network
    foodweb_network = nx.Graph()
    foodweb_network.add_nodes_from(consumers)
    foodweb_network.add_nodes_from(resources)
    for con,res in iter_product(consumers, resources):
        if foodweb_matrix.loc[con,res]:
            foodweb_network.add_edge(con,res)
    
    G = foodweb_network
    H = nx.induced_subgraph(G, max(nx.connected_components(G), key=len)).copy()

    if H.number_of_nodes() < min_nodes:
        continue

    dgm = rng.diagram(H)

    print('ID: ', file.split('.')[0])
    print('#Nodes in graph: ', len(G))
    print('#Nodes in LCC:   ', len(H))
    print('ring-score:      ', round(dgm.score, 3))
    print()

### Analysis

In [ ]:
foodweb_matrix = pd.read_csv(f"{FOODWEB_FOLDER}/M_SD_034.csv", index_col=0)
consumers = foodweb_matrix.index
resources = foodweb_matrix.columns

#  Construct network
G_food = nx.Graph()
G_food.add_nodes_from(consumers)
G_food.add_nodes_from(resources)
for con,res in iter_product(consumers, resources):
    if foodweb_matrix.loc[con,res]:
        G_food.add_edge(con,res)

pos = nx.spring_layout(G_food)

In [ ]:
plot_nx(G_food, pos=pos, node_alpha=0.5)

In [ ]:
degree_sequence = sorted([d for n, d in G_food.degree()])
degreeCount = Counter(degree_sequence)
degs, cnts = zip(*degreeCount.items())

deg_fig, ax = plt.subplots(figsize=(8,6))
deg_fig.patch.set_alpha(0)
ax.patch.set_alpha(0)

ax.bar(degs, cnts, width=1, color=sblue, linewidth=0)

ax.tick_params(axis='both', which='major', labelsize=24)

ax.spines['left'].set_linewidth(2.5)
ax.spines['left'].set_color(sblack2)

ax.spines['bottom'].set_linewidth(2.5)
ax.spines['bottom'].set_color(sblack2)

In [ ]:
degree_sequence = sorted([d for n, d in G_food.degree()])

deg_fig, ax = plt.subplots(figsize=(8,6))
deg_fig.patch.set_alpha(0)
ax.patch.set_alpha(0)

ax.hist(degree_sequence, bins=25)

ax.tick_params(axis='both', which='major', labelsize=24)

ax.spines['left'].set_linewidth(2.5)
ax.spines['left'].set_color(sblack2)

ax.spines['bottom'].set_linewidth(2.5)
ax.spines['bottom'].set_color(sblack2)

plt.savefig("/Users/markusyoussef/Desktop/food_distribution.png")

## Consumer networks

In [ ]:
# -------------------------
# Extract foodweb networks
# -------------------------


for file in os.listdir(FOODWEB_FOLDER):
    if not file.endswith('csv') or file == 'references.csv':
        continue
    
 #  Load table 
    foodweb_matrix = pd.read_csv(f"{FOODWEB_FOLDER}/{file}", index_col=0)
    consumers = foodweb_matrix.index

 #  Construct network
    consumer_network = nx.Graph()
    consumer_network.add_nodes_from(consumers)
    
    for con1,con2 in combinations(consumers,2):
        if (foodweb_matrix.loc[con1] + foodweb_matrix.loc[con2]).any():
            consumer_network.add_edge(con1, con2)
    
    G = consumer_network
    H = nx.induced_subgraph(G, max(nx.connected_components(G), key=len)).copy()

    if H.number_of_nodes() < min_nodes or nx.density(H) == 1:
        continue

    dgm = rng.diagram(H)

    print('ID: ', file.split('.')[0])
    print('#Nodes in graph: ', len(G))
    print('#Nodes in LCC:   ', len(H))
    print('ring-score:      ', round(dgm.score, 3))
    print()

## Resources networks

In [ ]:
# -------------------------
# Extract foodweb networks
# -------------------------


for file in os.listdir(FOODWEB_FOLDER):
    if not file.endswith('csv') or file == 'references.csv':
        continue
    
 #  Load table 
    foodweb_matrix = pd.read_csv(f"{FOODWEB_FOLDER}/{file}", index_col=0)
    resources = foodweb_matrix.columns

 #  Construct network
    resource_network = nx.Graph()
    resource_network.add_nodes_from(resources)
    
    for res1,res2 in combinations(resources,2):
        if (foodweb_matrix[res1] + foodweb_matrix[res2]).any():
            consumer_network.add_edge(res1, res2)
    
    G = resource_network
    H = nx.induced_subgraph(G, max(nx.connected_components(G), key=len)).copy()

    if H.number_of_nodes() < min_nodes or nx.density(H) == 1:
        continue
    break

    dgm = rng.diagram(H)

    print('ID: ', file.split('.')[0])
    print('#Nodes in graph: ', len(G))
    print('#Nodes in LCC:   ', len(H))
    print('ring-score:      ', round(dgm.score, 3))
    print()

## Genes
<center><h3 style="color:darkred"> >>> CAUTION - Big Data! <<<  </h3></center><p>

*Uncomment lines if the data is not produced yet.*

### Network processing

In [ ]:
c = 0.95

# C = np.genfromtxt('data/biological_networks/genes/gene_corr.txt')
# D = 1-np.abs(C)
# A = np.where(np.abs(C)>c,1,0)
# np.fill_diagonal(A,0)
# G = nx.from_numpy_array(A)
# G = max(nx.connected_component_subgraphs(G), key=len)
# G = nx.convert_node_labels_to_integers(G)

# for _, _, data in G.edges(data=True): data.pop('weight',None)
# nx.write_edgelist(G, f'data/biological_networks/genes/gene_c{c:.2f}_edgelist.csv', data=False)

### Network visuals - this might take a while ...    
&nbsp;&nbsp;&nbsp;&nbsp;(~15min on my computer)

*Load network for rerunning the visuals.*

In [ ]:
G = nx.read_edgelist(f'data/biological_networks/genes/gene_c{c:.2f}_edgelist.csv', nodetype=int, data=False)

*Precompute/load layout positions.*

In [ ]:
# pos = nx.spring_layout(G)
# with open(f'data/biological_networks/genes/gene_c{c:.2f}_pos.pkl', 'wb') as f:
#     pickle.dump(pos, f)

In [ ]:
with open(f'data/biological_networks/genes/gene_c{c:.2f}_pos.pkl', 'rb') as f:
    pos = pickle.load(f)

*Plotting the network; this is the most time-consuming step. You might wanna save it right away.*

In [ ]:
# pos = comb(pos)      # moves sticky-out edges closer to the center
# plot_nx(G, pos=pos)

*Save plot with precomputed layout positions.*

In [ ]:
plot_nx(G, path=f'data/biological_networks/genes/gene_c{c:.2f}_nx.pdf',  silence=True, pos=pos)

In [ ]:
mode = 'bar'

degree_sequence = sorted([d for n, d in G.degree()])
degreeCount = Counter(degree_sequence)
degs, cnts = zip(*degreeCount.items())

deg_fig, ax = plt.subplots(figsize=(8,6))
deg_fig.patch.set_alpha(0)
ax.patch.set_alpha(0)

eval(f"ax.{mode}(degs, cnts, width=1, color=sblue, linewidth=0)");

ax.tick_params(axis='both', which='major', labelsize=24)

ax.spines['left'].set_linewidth(2.5)
ax.spines['left'].set_color(sblack2)

ax.spines['bottom'].set_linewidth(2.5)
ax.spines['bottom'].set_color(sblack2)

deg_fig.savefig(f'google_drive/genes/gene_c{c:.2f}_deg_{mode}.pdf', transparent=True)

In [ ]:
deg_fig.savefig(f'data/biological_networks/genes/gene_c{c:.2f}_deg.pdf', transparent=True)

### Calculating diagrams - you probably won't be able to calculate this on a regular computer...    
&nbsp;&nbsp; (Some of these calculations require up to 256GB RAM and can take up to 24h.)

In [ ]:
# rng.induce_toa(G, split=10)
# nx.write_edgelist(G, f'data/biological_networks/genes/gene_c{c:.2f}_weightlist.csv')

In [ ]:
# dgm = rng.diagram(G, toa='toa')
# dgm.save(f'data/biological_networks/genes/gene_c{c:.2f}_dgm.csv', )

In [ ]:
# dgm = rng.ripser(D)
# dgm.save('data/biological_networks/genes/gene_dgm.csv')

In [ ]:
dgm = rng.load_dgm('data/biological_networks/genes/gene_dgm.csv')
dgm_c = rng.load_dgm(f'data/biological_networks/genes/gene_c{c:.2f}_dgm.csv')

In [ ]:
# persistence visuals
fig, axes = plt.subplots(2, 2, figsize=(14,13))

axes[0,0].set_title('Full Distance Matrix', fontsize=16)
axes[0,1].set_title(f'Cutoff at c={c:.2f}', fontsize=16)

plot_bar(dgm,   ax=axes[0,0], cap=50)
plot_bar(dgm_c, ax=axes[0,1], cap=50)
plot_dgm(dgm,   ax=axes[1,0])
plot_dgm(dgm_c, ax=axes[1,1])

axes[0,0].annotate(f'score = {dgm.GGS  :.3f}', xy=(0.7, 0.85), xycoords='axes fraction', fontsize=14);
axes[0,1].annotate(f'score = {dgm_c.GGS:.3f}', xy=(0.7, 0.85), xycoords='axes fraction', fontsize=14);
axes[1,0].annotate(f'score = {dgm.GGS  :.3f}', xy=(0.7, 0.15), xycoords='axes fraction', fontsize=14);
axes[1,1].annotate(f'score = {dgm_c.GGS:.3f}', xy=(0.7, 0.15), xycoords='axes fraction', fontsize=14);

### Write summary - this might take a while ...
&nbsp;&nbsp;&nbsp;&nbsp;(~12h/34h on my computer)

In [ ]:
write_summary(G, dgm=dgm_c, path=f'data/biological_networks/genes/gene_c{c:.2f}_summary.html')

### Save persistent visuals

In [ ]:
plot_dgm(dgm, path=f'data/biological_networks/genes/gene_dgm.pdf', silence=True)
plot_bar(dgm, path=f'data/biological_networks/genes/gene_bar.pdf', silence=True, cap=30)

plot_dgm(dgm_c, path=f'data/biological_networks/genes/gene_c{c:.2f}_dgm.pdf', silence=True)
plot_bar(dgm_c, path=f'data/biological_networks/genes/gene_c{c:.2f}_bar.pdf', silence=True, cap=30)

## Soil

### MSE

In [ ]:
# network preprocessing
d = 90

D = np.genfromtxt('data/biological_networks/soil/soil_gray_mse.txt')
A = np.where(np.abs(D)>d,0,1)
np.fill_diagonal(A,0)
G = nx.from_numpy_array(A)
# G = max(nx.connected_component_subgraphs(G), key=len)
# G = nx.convert_node_labels_to_integers(G)

for _, _, data in G.edges(data=True): data.pop('weight',None)
    
pos = nx.spring_layout(G, iterations=1000)

In [ ]:
# network visuals
plot_nx(G, pos=pos)

In [ ]:
mode = 'loglog'

degree_sequence = sorted([d for n, d in G.degree()])
degreeCount = Counter(degree_sequence)
degs, cnts = zip(*degreeCount.items())

deg_fig, ax = plt.subplots(figsize=(8,6))
deg_fig.patch.set_alpha(0)
ax.patch.set_alpha(0)

eval(f"ax.{mode}(degs, cnts, '*', color=sblue)");

ax.tick_params(axis='both', which='major', labelsize=24)

ax.spines['left'].set_linewidth(2.5)
ax.spines['left'].set_color(sblack2)

ax.spines['bottom'].set_linewidth(2.5)
ax.spines['bottom'].set_color(sblack2)

deg_fig.savefig(f'google_drive/soil/soil_mse_d{d:.2f}deg_{mode}.pdf', transparent=True)

In [ ]:
# calclulating diagrams
dgm   = rng.ripser(D)

rng.induce_toa(G)
dgm_d = rng.diagram(G, toa='toa')

In [ ]:
# add some vacuous points to the diagram to make the barplot look nicer
for i in range(10): dgm_d.append((0,0))

In [ ]:
# persistence visuals
fig, axes = plt.subplots(2, 2, figsize=(14,13))

axes[0,0].set_title('Full Distance Matrix', fontsize=16)
axes[0,1].set_title(f'Cutoff at d={d:.2f}', fontsize=16)

plot_bar(dgm,   ax=axes[0,0], cap=50)
plot_bar(dgm_d, ax=axes[0,1], cap=50)
plot_dgm(dgm,   ax=axes[1,0])
plot_dgm(dgm_d, ax=axes[1,1])

axes[0,0].annotate(f'score = {dgm.GGS  :.3f}', xy=(0.7, 0.875), xycoords='axes fraction', fontsize=14);
axes[0,1].annotate(f'score = {dgm_d.GGS:.3f}', xy=(0.7, 0.875), xycoords='axes fraction', fontsize=14);
axes[1,0].annotate(f'score = {dgm.GGS  :.3f}', xy=(0.7, 0.125), xycoords='axes fraction', fontsize=14);
axes[1,1].annotate(f'score = {dgm_d.GGS:.3f}', xy=(0.7, 0.125), xycoords='axes fraction', fontsize=14);

### Save data

In [ ]:
# save data
dgm.save(  f'data/biological_networks/soil/soil_mse_dgm.csv')
dgm_d.save(f'data/biological_networks/soil/soil_mse_d{d:.2f}_dgm.csv')

nx.write_edgelist(G, f'data/biological_networks/soil/soil_mse_d{d:.2f}_weightlist.csv')
nx.write_edgelist(G, f'data/biological_networks/soil/soil_mse_d{d:.2f}_edgelist.csv', data=False)

write_summary(G, dgm_d, path=f'data/biological_networks/soil/soil_mse_d{d:.2f}_summary.html')

In [ ]:
# save visuals
plot_dgm(dgm, path=f'data/biological_networks/soil/soil_mse_dgm.pdf', silence=True)
plot_bar(dgm, path=f'data/biological_networks/soil/soil_mse_bar.pdf', silence=True, cap=30)

plot_dgm(dgm_d, path=f'data/biological_networks/soil/soil_mse_d{d:.2f}_dgm.pdf', silence=True)
plot_bar(dgm_d, path=f'data/biological_networks/soil/soil_mse_d{d:.2f}_bar.pdf', silence=True, cap=30)
plot_nx(G,      path=f'data/biological_networks/soil/soil_mse_d{d:.2f}_nx.pdf',  silence=True, pos=pos)

deg_fig.savefig(f'data/biological_networks/soil/soil_mse_d{d:.2f}_deg.pdf', transparent=True)

### SSIM

In [ ]:
# network preprocessing
c = 0.95

C = np.genfromtxt('data/biological_networks/soil/soil_gray_ssim.txt')
D = 1-np.abs(C)
A = np.where(np.abs(C)>c,1,0)
np.fill_diagonal(A,0)
G = nx.from_numpy_array(A)
# G = max(nx.connected_component_subgraphs(G), key=len)
# G = nx.convert_node_labels_to_integers(G)

for _, _, data in G.edges(data=True): data.pop('weight',None)
    
pos = nx.spring_layout(G)

In [ ]:
# network visuals
plot_nx(G, pos=pos)

In [ ]:
# calclulating diagrams
dgm   = rng.ripser(D)

rng.induce_toa(G)
dgm_c = rng.diagram(G, toa='toa')

In [ ]:
# add some vacuous points to the diagram to make the barplot look nicer
for i in range(10): dgm.append((0,0))
for i in range(10): dgm_c.append((0,0))

In [ ]:
# persistence visuals
fig, axes = plt.subplots(2, 2, figsize=(14,13))

axes[0,0].set_title('Full Distance Matrix', fontsize=16)
axes[0,1].set_title(f'Cutoff at c={c:.2f}', fontsize=16)

plot_bar(dgm,   ax=axes[0,0], cap=50)
plot_bar(dgm_c, ax=axes[0,1], cap=50)
plot_dgm(dgm,   ax=axes[1,0])
plot_dgm(dgm_c, ax=axes[1,1])

axes[0,0].annotate(f'score = {dgm.GGS  :.3f}', xy=(0.7, 0.875), xycoords='axes fraction', fontsize=14);
axes[0,1].annotate(f'score = {dgm_c.GGS:.3f}', xy=(0.7, 0.875), xycoords='axes fraction', fontsize=14);
axes[1,0].annotate(f'score = {dgm.GGS  :.3f}', xy=(0.7, 0.125), xycoords='axes fraction', fontsize=14);
axes[1,1].annotate(f'score = {dgm_c.GGS:.3f}', xy=(0.7, 0.125), xycoords='axes fraction', fontsize=14);

### Save Data

In [ ]:
# save data
dgm.save(  f'data/biological_networks/soil/soil_ssim_dgm.csv')
dgm_c.save(f'data/biological_networks/soil/soil_ssim_c{c:.2f}_dgm.csv')

nx.write_edgelist(G, f'data/biological_networks/soil/soil_ssim_c{c:.2f}_weightlist.csv')
nx.write_edgelist(G, f'data/biological_networks/soil/soil_ssim_c{c:.2f}_edgelist.csv', data=False)

write_summary(G, dgm_c, path=f'data/biological_networks/soil/soil_ssim_c{c:.2f}_summary.html')

In [ ]:
# save visuals
plot_dgm(dgm, path=f'data/biological_networks/soil/soil_ssim_dgm.pdf', silence=True)
plot_bar(dgm, path=f'data/biological_networks/soil/soil_ssim_bar.pdf', silence=True, cap=50)

plot_dgm(dgm_c, path=f'data/biological_networks/soil/soil_ssim_c{c:.2f}_dgm.pdf', silence=True)
plot_bar(dgm_c, path=f'data/biological_networks/soil/soil_ssim_c{c:.2f}_bar.pdf', silence=True, cap=50)
plot_nx(G,      path=f'data/biological_networks/soil/soil_ssim_c{c:.2f}_nx.pdf',  silence=True, pos=pos)

## Interferons 

In [ ]:
# network preprocessing
G = nx.read_edgelist('data/biological_networks/IFNs/IFN_edgelist.csv')
G = G.to_undirected()
G = max(nx.connected_component_subgraphs(G), key=len)
G = nx.convert_node_labels_to_integers(G)

pos = nx.spring_layout(G, iterations=1000)

In [ ]:
# network visuals
# pos = comb(pos) # moves sticky-out edges closer to the center
plot_nx(G, pos=pos, node_alpha=0.8, edge_alpha=0.6)

In [ ]:
mode = 'bar'

degree_sequence = sorted([d for n, d in G.degree()])
degreeCount = Counter(degree_sequence)
degs, cnts = zip(*degreeCount.items())

deg_fig, ax = plt.subplots(figsize=(8,6))
deg_fig.patch.set_alpha(0)
ax.patch.set_alpha(0)

eval(f"ax.{mode}(degs, cnts, width=1, color=sblue)");

ax.tick_params(axis='both', which='major', labelsize=24)

ax.spines['left'].set_linewidth(2.5)
ax.spines['left'].set_color(sblack2)

ax.spines['bottom'].set_linewidth(2.5)
ax.spines['bottom'].set_color(sblack2)

deg_fig.savefig(f'google_drive/IFNs/IFN_deg_{mode}.pdf', transparent=True)

In [ ]:
# calclulating diagrams
rng.induce_toa(G)
dgm = rng.diagram(G, toa='toa')

In [ ]:
# persistence visuals
fig, axes = plt.subplots(1,2, figsize=(16,8))

axes[0].set_title('Barplot', fontsize=16)
axes[1].set_title('Persistence plot', fontsize=16)

plot_bar(dgm, ax=axes[0], cap=50)
plot_dgm(dgm, ax=axes[1])

axes[0].annotate(f'score = {dgm.GGS  :.3f}', xy=(0.7, 0.875), xycoords='axes fraction', fontsize=14);
axes[1].annotate(f'score = {dgm.GGS  :.3f}', xy=(0.7, 0.125), xycoords='axes fraction', fontsize=14);

### Save data

In [ ]:
# save data
dgm.save(  f'data/biological_networks/IFNs/IFN_dgm.csv')
nx.write_edgelist(G, f'data/biological_networks/IFNs/IFN_weightlist.csv')
write_summary(G, dgm, path=f'data/biological_networks/IFNs/IFN_summary.html')

In [ ]:
# save visuals
plot_dgm(dgm, path=f'data/biological_networks/IFNs/IFN_dgm.pdf', silence=True)
plot_bar(dgm, path=f'data/biological_networks/IFNs/IFN_bar.pdf', silence=True, cap=30)
plot_nx(G,    path=f'data/biological_networks/IFNs/IFN_nx.pdf',  silence=True, 
        pos=pos, node_alpha=0.8, edge_alpha=0.6)

deg_fig.savefig(f'data/biological_networks/IFNs/IFN_deg.pdf', transparent=True)